In [ ]:
%reload_ext tgpy
import tgpy as tg

In [ ]:
npriors = 2 # 1, 2, 3
ngroups = 1 # any positive int
nparams = 300 # number of chains
niters = 300 # number of iterations

### Create Priors

In [ ]:
prior0 = tg.TgPriorUnivariate('prior0', ['g{}'.format(i) for i in range(ngroups)], dim=nparams, 
                               low=-4, high=4, alpha=5, beta=5) # simetrico
prior1 = tg.TgPriorUnivariate('prior1', ['g{}'.format(i) for i in range(ngroups)], dim=nparams, 
                               low=-1, high=10, alpha=2, beta=3) # masa a la izquierda
prior2 = tg.TgPriorUnivariate('prior2', ['g{}'.format(i) for i in range(ngroups)], dim=nparams, 
                               low=-10, high=1, alpha=2, beta=1) # masa a la derecha

if npriors == 1:
    k = tg.DummyKernel(prior0)
elif npriors == 2:
    k = tg.DummyKernel(prior0, prior1)
else:
    k = tg.DummyKernel(prior0, prior1, prior2)


cov = tg.CovarianceTransport(k, noise=None)
tgp = tg.TGP([cov], dt=None, annealing=0)

tgp.sample_priors()
tgp.plot_priors()

In [ ]:
prior01 = tg.TgPriorBivariate('prior01', [prior0, prior1], r=3, dim=nparams)
prior01.sample_params()
prior01.logp()
prior01.plot()

k = tg.DummyKernel(prior01)

cov = tg.CovarianceTransport(k, noise=None)
tgp01 = tg.TGP([cov], dt=None, annealing=0)

learning01 = tg.TgLearning(tgp01, lr=0.1)
len(learning01.parameters_list), learning01.parameters_list[0].shape

### SGD

In [ ]:
learning = tg.TgLearning(tgp, lr=0.1)
learning.execute_sgd(niters)
tgp.plot_priors(kde=False)

In [ ]:
learning = tg.TgLearning(tgp, lr=0.1)
learning.execute_svgd(niters)
tgp.plot_priors(kde=False)

In [ ]:
learning01 = tg.TgLearning(tgp01, lr=0.1)
learning01.execute_sgd(niters)
tgp01.plot_priors()

### SVGD

In [ ]:
learning01 = tg.TgLearning(tgp01, lr=0.01)
learning01.execute_svgd(niters)
tgp01.plot_priors()

In [ ]:
tgp01.sample_priors()
tgp01.plot_priors()